
<img src='picture/670_1.png'>



In [2]:
import tensorflow as tf
from datetime import datetime
import time
import math

In [3]:
slim = tf.contrib.slim
#trunc_normal:产生截断的正态分布
trunc_normal = lambda stddev: tf.truncated_normal_initializer(0.0, stddev)

num_batches = 100

In [5]:
'''
inception_arg_scope：用来生成网络中经常用到的函数的默认参数，比如卷积的激活函数、权重初始化方式、标准化器等。
L2正则的weight_decay默认值为0.00004，标准差stddev默认值为0.1，参数batch_norm_var_collection默认值为moving_vars
'''
def inception_v3_arg_scope(weight_decay=0.00004,
                               stddev=0.1,
                               batch_norm_var_collection='moving_vars'): 
    '''
    参数字典
    '''
    batch_norm_params = {
        'decay': 0.9997,    #衰减系数
        'epsilon': 0.001,
        'updates_collections': tf.GraphKeys.UPDATE_OPS,
        'variables_collections': {
            'beta': None,
            'gama': None,
            'moving_mean': [batch_norm_var_collection],
            'moving_variance': [batch_norm_var_collection],
        }
    } 
    '''
    slim.arg_scope,可以给函数的参数自动赋予某些默认值
    '''
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                        weights_regularizer=slim.l2_regularizer(weight_decay)):
        with slim.arg_scope(
            [slim.conv2d],
            weights_initializer=tf.truncated_normal_initializer(stddev=stddev),
            activation_fn=tf.nn.relu,
            normalizer_fn=slim.batch_norm,
            normalizer_params=batch_norm_params
        ) as sc:
            return sc 

In [37]:
'''
inception_v3_base:生成Inception V3网络的卷积部分
参数inputs为输入的图片数据的tensor，scope为包含了函数默认参数的环境。
输出为: 
'''
def inception_v3_base(inputs, scope=None):
    end_points = {}
    with tf.variable_scope(scope, 'InceptionV3', [inputs]):
        with slim.arg_scope([slim.cov2d, slim.max_pool2d, slim.avg_pool2d], 
                            stride=1, padding='VALID') :#为函数设置默认值
            net = slim.conv2d(inputs, 32, [3, 3], stride=2, scope='Conv2d_1a_3x3')
            net = slim.conv2d(net, 32, [3, 3], scope='Conv2d_2a_3x3')
            net = slim.conv2d(net, 64, [3, 3], padding='SAME', scope='Conv2d_2b_3x3')
            net = slim.max_pool2d(net, [3, 3], stride=2, scope='Max_pool_3a_3x3')
            net = slim.conv2d(net, 80, [1, 1], scope='Conv2d_3b_1x1')
            net = slim.conv2d(net, 192, [3, 3], stride=2, scope='Conv2d_4a_3x3')
            net = slim.max_pool2d(net, [3, 3], stride=2, scope='MaxPool_5a_3x3')
        
    """
    第1个Inception模块组， 包括三个模块 
    """
    with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d],
                       stride=1, padding='SAME'):
        # 第一个Inception Module
        with tf.variable_scope('Mixed_5b'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 64, [1, 1], scope='Covn2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 48, [1, 1], scope='Conv2d_0a_1x1')
                branch_1 = slim.conv2d(branch_1, 64, [5, 5], scope='Conv2d_0b_5x5')
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
                branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0b_3x3')
                branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0c_3x3')
            with tf.variable_scope('Branch_3'):
                branch_3 = slim.avg_pool2d(net, 64, [3, 3], scope='AvgPool_0a_3x3')
                branch_3 = slim.conv2d(branch_3, 32, [1, 1], scope='Conv2d_0b_1x1') 
             #输出通道数为64+64+96+32=256，padding是'SAME'，输出为35*35*256
            net = tf.concat([branch_0, branch_1, branch_2, branch_3], axis=3)
        
        # 第二个 Inception Module
        with tf.variable_scope('Mixed_5c'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 48, [1, 1], scope='Conv2d_0a_1x1')
                branch_1 = slim.conv2d(branch_1, 64, [5, 5], scope='Conv2d_0b_5x5')
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
                branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0b_3x3')
                branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0c_3x3')
            with tf.variable_scope('Branch_3'):
                branch_3 = slim.avg_pool2d(net, 64, [3, 3], scope='AvgPool_0a_3x3')
                branch_3 = slim.conv2d(branch_3, 64, [1, 1], scope='Conv2d_0b_1x1') 
                #输出通道数为64+64+96+64=288，padding是'SAME'，输出为35*35*288
                net = tf.concat([branch_0, branch_1, branch_2, branch_3], axis=3)
        
        # 第三个 Inception Module
        with tf.variable_scope('Mixed_5d'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 48, [1, 1], scope='Conv2d_0a_1x1')
                branch_1 = slim.conv2d(branch_1, 64, [5, 5], scope='Conv2d_0b_5x5')
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
                branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0b_3x3')
                branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0c_3x3')
            with tf.variable_scope('Branch_3'):
                branch_3 = slim.avg_pool2d(net, 64, [3, 3], scope='AvgPool_0a_3x3')
                branch_3 = slim.conv2d(branch_3, 64, [1, 1], scope='Conv2d_0b_1x1') 
                #输出通道数为64+64+96+64=288，padding是'SAME'，输出为35*35*288
                net = tf.concat([branch_0, branch_1, branch_2, branch_3], axis=3)
         
    """
    第2个Inception模块组， 包括5个模块 
    """
    # 第1个Inception Module
    with tf.variable_scope('Mixed_6a'):
        with tf.variable_scope('Branch_0'):
            #图片缩小为17x17
            branch_0 = slim.conv2d(net, 384, [3, 3], stride=2, padding='VALID', scope='Conv2d_1a_3x2')
        with tf.variable_scope('Branch_1'):
            branch_1 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
            branch_1 = slim.conv2d(branch_1, 96, [3, 3], scope='Conv2d_1b_3x3')
            branch_1 = slim.conv2d(branch_1, 96, [3, 3], stride=2, padding='VALID',scope='Conv2d_1c_3x3')
        with tf.variable_scope('Branch_2'):
            branch_2 = slim.max_pool2d(net, [1, 1], stride=2, padding='VALID', scope='MaxPool_1a_3x3')
        #输出通道数为384+96+256=768，输出尺寸为17*17*768
        net = tf.concat([branch_0, branch_1, branch_2], axis=3)
    
    #后4个Module中都用到了'Factorization into small convolutions'
    # 第2个Inception Module
    with tf.variable_scope("Mixed_6b"):
        with tf.variable_scope('Branch_0'): 
            branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_2a_1x1')
        with tf.variable_scope('Branch_1'):
            branch_1 = slim.conv2d(net, 128, [1, 1], scope='Conv2d_2a_1x1')
            branch_1 = slim.conv2d(branch_1, 128, [3, 3], scope='Conv2d_2b_3x3')
            branch_1 = slim.conv2d(branch_1, 192, [3, 3], scope='Conv2d_2c_3x3')
        with tf.variable_scope('Branch_2'):
            branch_2 = slim.conv2d(net, 128, [1, 1], scope='Conv2d_2a_3x3')
            branch_2 = slim.conv2d(branch_2, 128, [7, 1], scope='Conv2d_2b_7x1') 
            branch_2 = slim.conv2d(branch_2, 128, [1, 7], scope='Conv2d_2c_1x7') 
            branch_2 = slim.conv2d(branch_2, 192, [7, 1], scope='Conv2d_2d_7x1') 
            branch_2 = slim.conv2d(branch_2, 192, [1, 7], scope='Conv2d_2e_1x7')  
        with tf.variable_scope('Branch_3'):
            branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_2a_3x3')
            branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_2b_1x1')  
        #输出通道为192+192+192+192=768，输出尺寸为17*17*768
        net = tf.concat([branch_0, branch_1, branch_2, branch_3], axis=3)
        
    # 第3个Inception Module
    with tf.variable_scope("Mixed_6c"):
        with tf.variable_scope('Branch_0'): 
            branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_2a_1x1')
        with tf.variable_scope('Branch_1'):
            branch_1 = slim.conv2d(net, 160, [1, 1], scope='Conv2d_2a_1x1')
            branch_1 = slim.conv2d(branch_1, 160, [1, 7], scope='Conv2d_2b_3x3')
            branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_2c_3x3')
        with tf.variable_scope('Branch_2'):
            branch_2 = slim.conv2d(net, 160, [1, 1], scope='Conv2d_2a_3x3')
            branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_2b_7x1') 
            branch_2 = slim.conv2d(branch_2, 160, [1, 7], scope='Conv2d_2c_1x7') 
            branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_2d_7x1') 
            branch_2 = slim.conv2d(branch_2, 192, [1, 7], scope='Conv2d_2e_1x7')  
        with tf.variable_scope('Branch_3'):
            branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_2a_3x3')
            branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_2b_1x1')  
        #输出通道为192+192+192+192=768，输出尺寸为17*17*768
        net = tf.concat([branch_0, branch_1, branch_2, branch_3], axis=3)
        
    # 第4个Inception Module
    with tf.variable_scope("Mixed_6d"):
        with tf.variable_scope('Branch_0'): 
            branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_2a_1x1')
        with tf.variable_scope('Branch_1'):
            branch_1 = slim.conv2d(net, 160, [1, 1], scope='Conv2d_2a_1x1')
            branch_1 = slim.conv2d(branch_1, 160, [1, 7], scope='Conv2d_2b_3x3')
            branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_2c_3x3')
        with tf.variable_scope('Branch_2'):
            branch_2 = slim.conv2d(net, 160, [1, 1], scope='Conv2d_2a_3x3')
            branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_2b_7x1') 
            branch_2 = slim.conv2d(branch_2, 160, [1, 7], scope='Conv2d_2c_1x7') 
            branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_2d_7x1') 
            branch_2 = slim.conv2d(branch_2, 192, [1, 7], scope='Conv2d_2e_1x7')  
        with tf.variable_scope('Branch_3'):
            branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_2a_3x3')
            branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_2b_1x1')  
        #输出通道为192+192+192+192=768，输出尺寸为17*17*768
        net = tf.concat([branch_0, branch_1, branch_2, branch_3], axis=3)
        
    # 第5个Inception Module
    with tf.variable_scope("Mixed_6e"):
        with tf.variable_scope('Branch_0'): 
            branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_2a_1x1')
        with tf.variable_scope('Branch_1'):
            branch_1 = slim.conv2d(net, 160, [1, 1], scope='Conv2d_2a_1x1')
            branch_1 = slim.conv2d(branch_1, 160, [1, 7], scope='Conv2d_2b_3x3')
            branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_2c_3x3')
        with tf.variable_scope('Branch_2'):
            branch_2 = slim.conv2d(net, 160, [1, 1], scope='Conv2d_2a_3x3')
            branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_2b_7x1') 
            branch_2 = slim.conv2d(branch_2, 160, [1, 7], scope='Conv2d_2c_1x7') 
            branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_2d_7x1') 
            branch_2 = slim.conv2d(branch_2, 192, [1, 7], scope='Conv2d_2e_1x7')  
        with tf.variable_scope('Branch_3'):
            branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_2a_3x3')
            branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_2b_1x1')  
        #输出通道为192+192+192+192=768，输出尺寸为17*17*768
        net = tf.concat([branch_0, branch_1, branch_2, branch_3], axis=3)
    
    end_points['Mixed_6e'] = net
    
    '''
    第3个Inception模块组，包含3个Inception Module
    '''
    # 第1个Inception Module
    with tf.variable_scope("Mixed_7a"):
        with tf.variable_scope('Branch_0'): 
            branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_2a_1x1')
            branch_0 = slim.conv2d(branch_0, 320, [3, 3], stride=2, padding='VALID',scope='Conv2d_2b_3x3')
        with tf.variable_scope('Branch_1'):
            branch_1 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_2a_1x1')
            branch_1 = slim.conv2d(branch_1, 192, [1, 7], scope='Conv2d_2b_3x3')
            branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_2c_3x3')
            branch_1 = slim.conv2d(branch_1, 192, [3, 3], stride=2, padding='VALID',scope='Conv2d_2d_3x3')
        with tf.variable_scope('Branch_2'): 
            branch_2 = slim.max_pool2d(net, [3, 3], stride=2, padding='VALID',scope='AvgPool_2a_3x3')
            branch_2 = slim.conv2d(branch_2, 192, [1, 1], scope='Conv2d_2b_1x1')  
        #输出通道数为320+192+768=1280，输出尺寸为8*8*1280
        net = tf.concat([branch_0, branch_1, branch_2], axis=3)
    
    # 第2个Inception Module
    with tf.variable_scope("Mixed_7b"):
        with tf.variable_scope('Branch_0'): 
            branch_0 = slim.conv2d(net, 320, [1, 1], scope='Conv2d_3a_1x1') 
        with tf.variable_scope('Branch_1'):
            branch_1 = slim.conv2d(net, 384, [1, 1], scope='Conv2d_3a_1x1')
            branch_1 = tf.concat([
                slim.conv2d(branch_1, 192, [1, 3], scope='Conv2d_3b_1x3'),
                slim.conv2d(branch_1, 192, [3, 1], scope='Conv2d_3c_3x1')
            ], 3) 
        with tf.variable_scope('Branch_2'): 
            branch_2 = slim.conv2d(net, 448, [1, 1], scope='Conv2d_2a_3x3')
            branch_2 = slim.conv2d(branch_2, 384, [3, 3], scope='Conv2d_2b_3x3')  
            branch_2 = tf.concat([
                slim.conv2d(branch_2, 384, [1, 3], scope='Conv2d_3c_1x3') ,
                slim.conv2d(branch_2, 384, [3, 1], scope='Conv2d_3c_3x1')  
            ], 3)
        with tf.variable_scope('Branch_3'):
            branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_3a_3x3')
            branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_3b_1x1')
            
        # 输出通道数为320+768+768+192=2048，输出尺寸为8*8*2048
        net = tf.concat([branch_0, branch_1, branch_2, branch_3], axis=3)
        
    # 第3个Inception Module
    with tf.variable_scope("Mixed_7c"):
        with tf.variable_scope('Branch_0'): 
            branch_0 = slim.conv2d(net, 320, [1, 1], scope='Conv2d_3a_1x1') 
        with tf.variable_scope('Branch_1'):
            branch_1 = slim.conv2d(net, 384, [1, 1], scope='Conv2d_3a_1x1')
            branch_1 = tf.concat([
                slim.conv2d(branch_1, 384, [1, 3], scope='Conv2d_3b_1x3'),
                slim.conv2d(branch_1, 384, [3, 1], scope='Conv2d_3c_3x1')
            ], 3) 
        with tf.variable_scope('Branch_2'): 
            branch_2 = slim.conv2d(net, 448, [1, 1], scope='Conv2d_2a_3x3')
            branch_2 = slim.conv2d(branch_2, 384, [3, 3], scope='Conv2d_2b_3x3')  
            branch_2 = tf.concat([
                slim.conv2d(branch_2, 384, [1, 3], scope='Conv2d_3c_1x3') ,
                slim.conv2d(branch_2, 384, [3, 1], scope='Conv2d_3c_3x1')  
            ], 3)
        with tf.variable_scope('Branch_3'):
            branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_3a_3x3')
            branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_3b_1x1')    
        # 输出通道数为320+768+768+192=2048，输出尺寸为8*8*2048
        net = tf.concat([branch_0, branch_1, branch_2, branch_3], axis=3)
        
    return net, end_points
    
print("OK")

OK


In [41]:
'''
inception_v3:全局平均池化、Softmax和Auxiliary Logits
num_classes:分类数量
is_training:标志是否是训练过程，只有训练时Batch Normalization和Dropout才会被启用
dropout_keep_prob:Dropout所需保留节点的比例，默认为0.8
prediction_fn:最后用来进行分类的函数
spatial_squeeze:标志是否对输出进行squeeze操作（即去除维数为1的维度，如5x3x1转为5x3）
reuse:标志是否会对网络和Variable进行重复使用
scope:包含了默认参数的环境
'''
def inception_v3(inputs,
                 num_classes=100,
                 is_training=True,
                 dropout_keep_prob=0.8,
                 prediction_fn=slim.softmax,
                 spatial_squeeze=True,
                 reuse=None,
                 scope='InceptionV3'):
    with tf.variable_scope(scope, [inputs, num_classes], reuse=reuse) as scope:
        with slim.arg_scope([slim.batch_norm, slim.dropout], is_training=is_training):
            net, end_points = inception_v3_base(input, scope=scope)
        
        '''
        Auxiliary Logits作为辅助分类的节点，对分类结果预测有很大帮助
        '''
        with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d],
                           stride=1, padding='SAME'):
            aux_logits = end_points['Mixed_6e']
            with tf.variable_scope('AuxLogits'):
                aux_logits = slim.avg_pool2d(aux_logits, [5, 5], stride=3, padding='VALID',
                                            scope='AvgPool_1a_5x5')
                aux_logits = slim.conv2d(aux_logits, [1, 1], scope='Conv2d_1b_1x1')
                aux_logits = slim.conv2d(aux_logits, [5, 5], scope='Conv2d_1b_1x1')
                aux_logits = slim.conv2d(aux_logits, [5, 5],  weights_initializer=trunc_normal(0.01),
                                         padding='VALID', scope='Conv2d_1c_1x1')
                aux_logits = slim.conv2d(num_classes, [1, 1], activation_fn=None,
                                        normalizer_fn=None, weights_initializer=trunc_normal(0.001),
                                        scope='Conv2d_1d_1x1')
                if spatial_squeeze:
                    aux_logits = tf.squeeze(aux_logits, [1, 2], name='SpatialSqueeze')
                end_points['AuxLogits'] = aux_logits
                
        '''
        Logits进行正常的分类预测的逻辑
        '''
        with tf.variable_scope('Logits'):
            net = slim.avg_pool2d(net, [8, 8],
                                 padding='VALID', scope='AvgPool_1a_8x8')
            net = slim.dropout(net, keep_prob=dropout_keep_prob, scope='Dropout_1b')
            end_points['PreLogits'] = net
            logits = slim.conv2d(net, num_classes, [1, 1], 
                                activation_fn=None, normalizer_fn=None,
                                scope='Conv2d_1c_1x1')
            
            
            if spatial_squeeze:
                logits = tf.squeeze(locals, [1, 2], name='SpatialSqueeze')
            end_points['Logits'] = logits
            end_points['Predictions'] = prediction_fn(logits, scope='Predictions')
        return logits, end_points

                
print('OK')

OK
